### Multinomial Logistic Regression Model

In [2]:
# Dependencies
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pylab as pl
import statsmodels.api as sm

In [3]:
# Import csv
csvpath = os.path.join("data", "clean_data.csv")
data_df = pd.read_csv(csvpath)
data_df["id"] = data_df.index + 1
del data_df["Unnamed: 0"]
data_df.head()

,General Health,(%) of Population Under FPL,Eaten Fruits or Veggies Yesterday,Age Group,Race,Education,Insured,Exercise,Sex,Drinker,Smoker,id
0,4.0,2.0,2.0,4.0,2,3.0,1.0,1.0,0,0.0,0.0,1
1,4.0,3.0,2.0,2.0,4,2.0,1.0,1.0,0,0.0,0.0,2
2,4.0,2.0,1.0,5.0,1,4.0,1.0,1.0,1,1.0,1.0,3
3,2.0,1.0,2.0,5.0,1,2.0,1.0,1.0,0,0.0,0.0,4
4,3.0,1.0,2.0,4.0,1,4.0,1.0,1.0,0,1.0,1.0,5


In [4]:
# Modify DV for MNLM (Need to change base outcome to mid value)
# Ordinal relationship of values in this variable is not relevent in MNLM 
data_df["Unaltered Health"] = data_df["General Health"]
name_health = {"General Health": {5: "excellent", 4: "very good", 3: "good", 2: "fair", 1: "poor"}}
data_df.replace(name_health, inplace=True)
reposition_health = {"General Health": {"excellent" : 5, "very good" : 4, "fair" : 3, "poor" : 2, "good" : 1}}
data_df.replace(reposition_health, inplace=True)
data_df.head()

,General Health,(%) of Population Under FPL,Eaten Fruits or Veggies Yesterday,Age Group,Race,Education,Insured,Exercise,Sex,Drinker,Smoker,id,Unaltered Health
0,4,2.0,2.0,4.0,2,3.0,1.0,1.0,0,0.0,0.0,1,4.0
1,4,3.0,2.0,2.0,4,2.0,1.0,1.0,0,0.0,0.0,2,4.0
2,4,2.0,1.0,5.0,1,4.0,1.0,1.0,1,1.0,1.0,3,4.0
3,3,1.0,2.0,5.0,1,2.0,1.0,1.0,0,0.0,0.0,4,2.0
4,1,1.0,2.0,4.0,1,4.0,1.0,1.0,0,1.0,1.0,5,3.0


In [24]:
# Variable groups
y = data_df['General Health']

lifestyle = data_df.drop(['id', 'Unaltered Health', 'General Health','Sex', '(%) of Population Under FPL',
                          'Insured', 'Education', 'Race', 'Age Group'], axis = 1)

demo = data_df.drop(['id', 'Unaltered Health', 'General Health', 'Smoker', 'Sex',
                     'Eaten Fruits or Veggies Yesterday', "Exercise", "Drinker"], axis = 1)

In [32]:
# Model uses "good health" as the base outcome. Model variables can be added by removing them from the "X" list.
#Lifestyle Mlogit
ls_Xc = sm.add_constant(lifestyle)
ls_mlogit = sm.MNLogit(y, ls_Xc)
ls_fmlogit = ls_mlogit.fit()
ls_rel_odds = (np.exp(ls_fmlogit.params))
ls_rel_odds = (np.exp(ls_fmlogit.params))
ls_rel_odds.columns = ["Poor Health", "Fair Health", "Very Good Health", "Excellent Health"]
ls_rel_odds
for row in range(5):
    for col in range(4):
        ls_rel_odds.iloc[row, col] = (((ls_rel_odds.iloc[row, col]) - 1)*100)
ls_rel_odds.pct_change()

print(ls_fmlogit.summary())

Optimization terminated successfully.
         Current function value: 1.459542
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:         General Health   No. Observations:                 7913
Model:                        MNLogit   Df Residuals:                     7897
Method:                           MLE   Df Model:                           12
Date:                Tue, 12 Jun 2018   Pseudo R-squ.:                 0.03381
Time:                        19:53:08   Log-Likelihood:                -11549.
converged:                       True   LL-Null:                       -11954.
                                        LLR p-value:                2.696e-165
                 General Health=2       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                                -0.1552      0.191     -0

/anaconda3/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:3967: RuntimeWarning: invalid value encountered in sqrt
  bse = np.sqrt(np.diag(self.cov_params()))
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [33]:
# Show df of relative proportions
ls_rel_odds

,Poor Health,Fair Health,Very Good Health,Excellent Health
const,-14.374417,-20.246698,-81.045846,-89.630497
Eaten Fruits or Veggies Yesterday,-17.258887,-1.992897,45.169210,67.578440
Exercise,-58.803285,-28.542954,58.483574,65.482409
Drinker,-42.074798,-17.734620,28.739905,22.441285
Smoker,-42.074775,-17.734616,28.739896,22.441273


In [30]:
# Model uses "good health" as the base outcome. Model variables can be added by removing them from the "X" list.
# Demographic Mlogit
d_Xc = sm.add_constant(demo)
d_mlogit = sm.MNLogit(y, d_Xc)
d_fmlogit = d_mlogit.fit()
d_rel_odds = (np.exp(d_fmlogit.params))

d_rel_odds.columns = ["Poor Health", "Fair Health", "Very Good Health", "Excellent Health"]
d_rel_odds
for row in range(5):
    for col in range(4):
        d_rel_odds.iloc[row, col] = (((d_rel_odds.iloc[row, col]) - 1)*100)
d_rel_odds.pct_change()

print(d_fmlogit.summary())

Optimization terminated successfully.
         Current function value: 1.405941
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:         General Health   No. Observations:                 7913
Model:                        MNLogit   Df Residuals:                     7889
Method:                           MLE   Df Model:                           20
Date:                Tue, 12 Jun 2018   Pseudo R-squ.:                 0.06929
Time:                        19:52:15   Log-Likelihood:                -11125.
converged:                       True   LL-Null:                       -11954.
                                        LLR p-value:                     0.000
           General Health=2       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -4.2725      0.360    -11.859      0.000   

In [31]:
d_rel_odds

,Poor Health,Fair Health,Very Good Health,Excellent Health
const,-0.986054,-0.920560,-0.134761,-0.099829
(%) of Population Under FPL,0.183451,0.096205,-0.156403,-0.101328
Age Group,1.091219,0.502720,-0.180882,-0.232599
Race,0.097325,0.215708,-0.164010,-0.220809
Education,-0.309282,-0.205496,0.428149,0.433313
Insured,1.196236,1.271891,1.454157,1.196347


In [ ]:
# Plot relative percentages with y range from -1 to 1, possibly -2 to 2 depending on data
# Perhaps bar chart with positive bars green and negative bars red
# One plot for each category